<a href="https://colab.research.google.com/github/spandita2/stackhack_marketplace_prototype/blob/main/stackhack_marketplace_prototype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# StackHack 3.0 – Marketplace Prototype

## Problem Statement
# Build a marketplace app that enables customers to view, compare and order
# digital and physical products, along with tracking delivery.

# ## Features Implemented
# - Browse products by category and seller
# - Compare products side by side
# - Place orders
# - Track delivery (physical vs digital)
# - View customer reviews
# - AI-assisted recommendations (prototype)

# ## Tech Stack
# - Python
# - Gradio (UI)
# - Google Colab
# - GitHub

# ## Note
# This is a functional prototype built to demonstrate end-to-end marketplace
# workflows under hackathon constraints.

In [2]:
# STEP 2: Mock product database

products = [
    {
        "id": 1,
        "name": "Wireless Headphones",
        "category": "Electronics",
        "seller": "AudioHub",
        "type": "Physical",
        "price": 1999,
        "delivery_time": "3–5 days",
        "rating": 4.3,
        "reviews": [
            "Good sound quality",
            "Delivery was fast",
            "Worth the price"
        ]
    },
    {
        "id": 2,
        "name": "Python Programming Course",
        "category": "Education",
        "seller": "CodeAcademy",
        "type": "Digital",
        "price": 499,
        "delivery_time": "Instant access",
        "rating": 4.7,
        "reviews": [
            "Very beginner friendly",
            "Well explained",
            "Highly recommended"
        ]
    },
    {
        "id": 3,
        "name": "Smart Watch",
        "category": "Electronics",
        "seller": "TechWorld",
        "type": "Physical",
        "price": 2999,
        "delivery_time": "4–6 days",
        "rating": 4.1,
        "reviews": [
            "Nice design",
            "Battery could be better",
            "Accurate tracking"
        ]
    }
]


In [3]:
# STEP 3A: Helper functions for browsing and comparison

def get_product_names():
    """Return product names for dropdowns"""
    return [product["name"] for product in products]


def get_product_by_name(name):
    """Fetch product details using product name"""
    for product in products:
        if product["name"] == name:
            return product
    return None


def compare_products(product_name_1, product_name_2):
    """Compare two products side by side"""
    p1 = get_product_by_name(product_name_1)
    p2 = get_product_by_name(product_name_2)

    if not p1 or not p2:
        return "Please select two valid products."

    comparison = {
        "Product": [p1["name"], p2["name"]],
        "Seller": [p1["seller"], p2["seller"]],
        "Type": [p1["type"], p2["type"]],
        "Price (₹)": [p1["price"], p2["price"]],
        "Delivery Time": [p1["delivery_time"], p2["delivery_time"]],
        "Rating": [p1["rating"], p2["rating"]]
    }

    return comparison


In [4]:
compare_products("Wireless Headphones", "Smart Watch")

{'Product': ['Wireless Headphones', 'Smart Watch'],
 'Seller': ['AudioHub', 'TechWorld'],
 'Type': ['Physical', 'Physical'],
 'Price (₹)': [1999, 2999],
 'Delivery Time': ['3–5 days', '4–6 days'],
 'Rating': [4.3, 4.1]}

In [5]:
# STEP 4A: Order placement logic

def place_order(product_name):
    product = get_product_by_name(product_name)

    if not product:
        return "Invalid product selection."

    order = {
        "order_id": f"ORD{product['id']}123",
        "product": product["name"],
        "type": product["type"],
        "status": "Order Placed"
    }

    return order


In [6]:
# STEP 4B: Delivery tracking logic

def track_order(order):
    if order["type"] == "Digital":
        return [
            "Payment Successful",
            "Instant access granted"
        ]
    else:
        return [
            "Order Placed",
            "Packed",
            "Shipped",
            "Out for Delivery",
            "Delivered"
        ]


In [7]:
order = place_order("Wireless Headphones")
track_order(order)

['Order Placed', 'Packed', 'Shipped', 'Out for Delivery', 'Delivered']

In [8]:
order = place_order("Python Programming Course")
track_order(order)

['Payment Successful', 'Instant access granted']

In [9]:
!pip install gradio

In [12]:
import gradio as gr
import pandas as pd


def browse_products():
    df = pd.DataFrame(products)

    df = df[[
        "name",
        "category",
        "seller",
        "type",
        "price",
        "delivery_time",
        "rating"
    ]]

    df.columns = [
        "Product Name",
        "Category",
        "Seller",
        "Type",
        "Price (₹)",
        "Delivery Time",
        "Rating"
    ]

    return df




def compare_ui(p1, p2):
    result = compare_products(p1, p2)
    if isinstance(result, str):
        return result

    df = pd.DataFrame(result)

    summary = ""
    if df["Price (₹)"][0] < df["Price (₹)"][1]:
        summary = f"✅ {df['Product'][0]} is cheaper"
    elif df["Price (₹)"][0] > df["Price (₹)"][1]:
        summary = f"✅ {df['Product'][1]} is cheaper"
    else:
        summary = "✅ Both products cost the same"

    return df, summary



def order_and_track_ui(product_name):
    order = place_order(product_name)
    tracking_steps = track_order(order)

    output = f"🧾 Order Confirmed\n"
    output += f"Order ID: {order['order_id']}\n\n"
    output += "📦 Tracking Status:\n"

    for step in tracking_steps:
        output += f"✔ {step}\n"

    return output
custom_css = """
body {
    background-color: #0f172a;
}

h1, h2, h3 {
    color: #e5e7eb;
}

.gr-button {
    background-color: #2563eb;
    color: white;
    border-radius: 8px;
}

.gr-button:hover {
    background-color: #1d4ed8;
}

.gr-box, .gr-panel {
    border-radius: 12px;
}
"""



In [10]:
# STEP 6A: AI-assisted best value recommendation (rule-based)

def recommend_best_value():
    """
    Recommends the best value product based on
    highest rating per unit price.
    """
    best_product = None
    best_score = 0

    for product in products:
        score = product["rating"] / product["price"]
        if score > best_score:
            best_score = score
            best_product = product

    return {
        "Recommended Product": best_product["name"],
        "Seller": best_product["seller"],
        "Price (₹)": best_product["price"],
        "Rating": best_product["rating"],
        "Reason": "Highest rating-to-price ratio"
    }


In [11]:
def recommend_ui():
    recommendation = recommend_best_value()
    output = "🏆 AI Recommended Product\n\n"

    for key, value in recommendation.items():
        output += f"{key}: {value}\n"

    return output


In [28]:
def login_user(email):
    if email.strip() == "":
        return (
            None,
            "❌ Please enter an email.",
            gr.update(visible=True),
            gr.update(visible=False)
        )

    return (
        {"email": email},
        "⏳ Logging you in…",
        gr.update(visible=False),
        gr.update(visible=True)
    )



def logout_user():
    return (
        None,
        "🔐 You have been logged out.",
        gr.update(visible=True),    # show login
        gr.update(visible=False)    # hide marketplace
    )


In [29]:

with gr.Blocks(title="MarketMate – Smart Marketplace") as demo:

    gr.HTML("""
<style>
/* ---------- GLOBAL ---------- */
body {
    background: linear-gradient(135deg, #0f172a, #020617);
    font-family: 'Inter', system-ui, -apple-system, BlinkMacSystemFont;
}

/* ---------- HEADINGS ---------- */
h1 {
    font-size: 2.1rem;
    font-weight: 700;
    letter-spacing: -0.5px;
    color: #f8fafc;
}

h2, h3 {
    color: #e5e7eb;
    font-weight: 600;
}

p, label {
    color: #cbd5e1;
    font-size: 0.95rem;
}

/* ---------- GLASS PANELS ---------- */
.gr-box, .gr-panel {
    background: rgba(15, 23, 42, 0.75) !important;
    backdrop-filter: blur(10px);
    border-radius: 18px !important;
    border: 1px solid rgba(148, 163, 184, 0.15);
    box-shadow: 0 10px 30px rgba(0,0,0,0.25);
}

/* ---------- BUTTONS (PURPLE THEME) ---------- */
button {
    background: linear-gradient(135deg, #560f7d, #6d1ea3) !important;
    color: white !important;
    border-radius: 12px !important;
    padding: 10px 18px !important;
    font-weight: 600;
    border: none !important;
    transition: all 0.25s ease-in-out;
}

button:hover {
    transform: translateY(-1px);
    box-shadow: 0 8px 22px rgba(86, 15, 125, 0.55);
    background: linear-gradient(135deg, #4a0d6c, #560f7d) !important;
}

/* ---------- INPUTS ---------- */
textarea, input, select {
    background-color: rgba(2, 6, 23, 0.9) !important;
    color: #f1f5f9 !important;
    border-radius: 10px !important;
    border: 1px solid rgba(148, 163, 184, 0.25) !important;
    padding: 8px 10px;
}

/* ---------- TABLE ---------- */
table {
    background-color: rgba(2, 6, 23, 0.85) !important;
    color: #e5e7eb !important;
    border-radius: 14px;
    overflow: hidden;
}

thead {
    background: rgba(86, 15, 125, 0.35) !important;
}

th {
    color: #f8fafc !important;
    font-weight: 600;
}

/* ---------- TABS ---------- */
button[role="tab"] {
    background: transparent !important;
    color: #cbd5e1 !important;
    border-radius: 10px !important;
    padding: 8px 14px !important;
    font-weight: 500;
}

button[role="tab"][aria-selected="true"] {
    background: rgba(86, 15, 125, 0.35) !important;
    color: #f8fafc !important;
}

/* ---------- FOOTER ---------- */
.footer {
    text-align: center;
    color: #94a3b8;
    font-size: 0.85rem;
    margin-top: 30px;
}
</style>
""")

with gr.Blocks(title="MarketMate – Smart Marketplace") as demo:

    # ---------- AUTH STATE ----------
    user_state = gr.State(None)

    # ---------- LOGIN SECTION ----------
    login_section = gr.Column(visible=True)
    with login_section:
        gr.Markdown("## 🔐 Login to MarketMate")
        login_msg = gr.Markdown()
        email_input = gr.Textbox(
            label="Email address",
            placeholder="you@example.com"
        )
        login_btn = gr.Button("Login")

    # ---------- MARKETPLACE SECTION ----------
    marketplace_section = gr.Column(visible=False)
    with marketplace_section:
        gr.Markdown("# 🛒 MarketMate – Smart Marketplace")
        welcome_msg = gr.Markdown()

        logout_btn = gr.Button("Logout")

        gr.Markdown("---")

        # ⬇️ 🔥 PASTE YOUR EXISTING TABS BELOW 🔥 ⬇️

        with gr.Tab("📦 Browse Products"):
            browse_btn = gr.Button("View Products")
            browse_output = gr.Dataframe()
            browse_btn.click(browse_products, outputs=browse_output)

        with gr.Tab("🔍 Compare Products"):
            p1 = gr.Dropdown(get_product_names(), label="Select Product 1")
            p2 = gr.Dropdown(get_product_names(), label="Select Product 2")
            compare_btn = gr.Button("Compare Products")
            compare_output = gr.Dataframe()
            compare_summary = gr.Markdown()

            compare_btn.click(
                compare_ui,
                inputs=[p1, p2],
                outputs=[compare_output, compare_summary]
            )

        with gr.Tab("🧾 Order & Track"):
            product_select = gr.Dropdown(get_product_names(), label="Select Product")
            order_btn = gr.Button("Place Order & Track")
            order_output = gr.Textbox(lines=8)
            order_btn.click(
                order_and_track_ui,
                inputs=product_select,
                outputs=order_output
            )

        with gr.Tab("🤖 AI Recommendation"):
            recommend_btn = gr.Button("Get Best Value Product")
            recommend_output = gr.Textbox(lines=6)
            recommend_btn.click(recommend_ui, outputs=recommend_output)

    # ---------- BUTTON ACTIONS ----------
    login_btn.click(
        login_user,
        inputs=email_input,
        outputs=[
            user_state,
            login_msg,
            login_section,
            marketplace_section
        ]
    )

    logout_btn.click(
        logout_user,
        outputs=[
            user_state,
            login_msg,
            login_section,
            marketplace_section
        ]
    )

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://40ca336f6014082fc3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
